# Construction d'un dataset à features numériques intéressantes

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
d = pd.read_pickle("../Création du Dataset/datasetv2.pkl")
dataset = d.copy()

In [14]:
dataset.columns[:50]

Index(['Y', 'TOP0_CHAMP', 'TOP0_LVL', 'TOP0_TOTAL', 'TOP0_GWR', 'TOP0_VET',
       'TOP0_RANK', 'TOP0_HOT', 'TOP0_KDAG', 'TOP0_KDA', 'TOP0_WR', 'TOP0_NB',
       'TOP0_FILL', 'JGL0_CHAMP', 'JGL0_LVL', 'JGL0_TOTAL', 'JGL0_GWR',
       'JGL0_VET', 'JGL0_RANK', 'JGL0_HOT', 'JGL0_KDAG', 'JGL0_KDA', 'JGL0_WR',
       'JGL0_NB', 'JGL0_FILL', 'MID0_CHAMP', 'MID0_LVL', 'MID0_TOTAL',
       'MID0_GWR', 'MID0_VET', 'MID0_RANK', 'MID0_HOT', 'MID0_KDAG',
       'MID0_KDA', 'MID0_WR', 'MID0_NB', 'MID0_FILL', 'ADC0_CHAMP', 'ADC0_LVL',
       'ADC0_TOTAL', 'ADC0_GWR', 'ADC0_VET', 'ADC0_RANK', 'ADC0_HOT',
       'ADC0_KDAG', 'ADC0_KDA', 'ADC0_WR', 'ADC0_NB', 'ADC0_FILL',
       'SUP0_CHAMP'],
      dtype='object')

# Fonctions de conversion 

### Conversion du Rank
Convertir le rang en nombre de LP au dessus de Iron IV 0 LP

In [8]:
def add_rank_numerical_value_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_RANK','TOP1_RANK','JGL0_RANK','JGL1_RANK','MID0_RANK','MID1_RANK','ADC0_RANK','ADC1_RANK','SUP0_RANK','SUP1_RANK']
    rankOrder = ['IRON','BRONZE','SILVER','GOLD','PLATINUM','DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
    divisionOrder = ['IV','III','II','I']
    for col in listRankCols:
        rank = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['RANK','DIVISION','LP'])
        for i in range(len(rankOrder)):
            rank.loc[rank['RANK']==rankOrder[i], 'RANK'] = i

        for i in range(len(divisionOrder)):
            rank.loc[rank['DIVISION']==divisionOrder[i], 'DIVISION'] = i
        
        datasetOutput[col+'_NUM'] = rank['RANK']*400 + rank['DIVISION']*100 + rank['LP']

### Conversion des KDA/KDAG
Calcul du ratio (K+A)/D

In [7]:
def add_kda_ratio_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_KDA','TOP1_KDA','JGL0_KDA','JGL1_KDA','MID0_KDA','MID1_KDA','ADC0_KDA','ADC1_KDA','SUP0_KDA','SUP1_KDA']
    for col in listRankCols:
        kda = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetOutput[col+'_RATIO'] = (kda['K'] + kda['A']) / (kda['D']+1)
        kdag = pd.DataFrame(datasetInput[col+'G'].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetOutput[col+'G'+'_RATIO'] = (kdag['K'] + kdag['A']) / (kdag['D']+1)

### Conversion des booléens
Calcul d'un "score", qui représente le nombre de joueurs possédant la caractéristique

In [9]:
def add_bool_score_column(datasetInput,datasetOutput, boolvar):
    listCols = ['TOP0_','TOP1_','JGL0_','JGL1_','MID0_','MID1_','ADC0_','ADC1_','SUP0_','SUP1_']
    datasetOutput[boolvar+'_SCORE'] = 0
    for i in range(len(listCols)):
        if i%2==0:
            datasetOutput[boolvar+'_SCORE'] = datasetOutput[boolvar+'_SCORE'] + datasetInput[listCols[i]+boolvar]
        else:
            datasetOutput[boolvar+'_SCORE'] = datasetOutput[boolvar+'_SCORE'] - datasetInput[listCols[i]+boolvar]